In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


### **Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@sssadatabricksete.dfs.core.windows.net/customers")
display(df)

In [0]:
df= df.drop("_rescued_data")

In [0]:
 df = df.withColumn("domains", split(col("email"), "@")[1])

In [0]:
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()

In [0]:
df_gmail = df.filter(col("domains") == "gmail.com")
df_gmail.display()

In [0]:
df= df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df = df.drop('first_name','last_name')

In [0]:
df.display()

### **Data Writing in Silver**

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@sssadatabricksete.dfs.core.windows.net/customers")

In [0]:
%sql
create table if not exists databricks_cata.silver.customers_silver
using delta
location "abfss://silver@sssadatabricksete.dfs.core.windows.net/customers"